In [45]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as pyf
import json

spark = SparkSession.builder.getOrCreate()

questions_df = spark.read.json("../../data/questions.json" )
exploded_questions_df = questions_df\
    .selectExpr("explode(items) as item")\
    .select("item.*")
# exploded_questions_df.show()


answers_df = spark.read.json("../../data/answers.json" )
exploded_answers_df = answers_df \
    .selectExpr("explode(items) as item")\
    .select("item.*")

exploded_answers_df = exploded_answers_df.select([c for c in exploded_answers_df.columns if c not in {"owner", "tag"}])

minimal_questions = exploded_questions_df \
    .select(pyf.col("body").alias("question"), "question_id")

minimal_answers = exploded_answers_df \
    .filter(pyf.col("is_accepted") == True) \
    .select(pyf.col("body").alias("answer"), "question_id")

joined_df = minimal_questions \
    .join(minimal_answers, on="question_id", how="inner")

joined_list = joined_df.collect()
# for x in joined_list:
#     # Serializing json
#     json_object = json.dumps(x, indent=4)
    
#     # Writing to sample.json
#     with open("sample.json", "w") as outfile:
#         outfile.write(json_object)
# joined_df.write.options(mode="overwrite").json("output.json")

+-----------+-----+
|question_id|count|
+-----------+-----+
|   77561774|    1|
|   75323747|    1|
|   71496561|    1|
|   73212628|    1|
|   72644013|    1|
|   75217063|    1|
|   76077950|    1|
|   69575496|    1|
|   75525312|    1|
|   77881942|    1|
|   75984983|    1|
|   77522753|    1|
|   71759316|    1|
|   72642575|    1|
|   70105756|    1|
|   74895640|    1|
|   76207341|    1|
|   71486019|    1|
|   72359181|    1|
|   77440877|    1|
+-----------+-----+
only showing top 20 rows



24/09/12 14:13:07 WARN ParseMode: overwrite is not a valid parse mode. Using PERMISSIVE.
